### Exp5.2.L_LeedsDataset_Onlytasks_CompvsIncomp_Combined_Push_Complex_new
Author : @leopauly | cnlp@leeds.ac.uk <br>
Description : Studying the feature vectors in the feature space using Leeds Action Dataset

In [1]:
import os
from six.moves import xrange  
import PIL.Image as Image
import random
import numpy as np
import cv2
import time
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from skimage import io
import warnings
warnings.filterwarnings('ignore')

# Custom scripts
import lscript as lsp
import modelling as md

Using TensorFlow backend.


In [2]:
height=112 
width=112 
channel=3
crop_size=112

cluster_length=16
feature_size=8192 
nb_classes=2
saved_path='/nobackup/leopauly/S2l/'

batch_size=32
memory_batch_size_train=266
memory_batch_size_test=170
next_batch_start=0
sample_batch_size=10

####  Loading model

In [3]:
## Defining placeholders in tf for images and targets
x_image = tf.placeholder(tf.float32, [None, 16,height,width,channel],name='x') 
y_true = tf.placeholder(tf.float32, [None, nb_classes],name='y_true')
y_true_cls = tf.placeholder(tf.int64, [None],name='y_true_cls')

model_keras = md.C3D_ucf101_training_model_tf(summary=True)
out=model_keras(x_image)
y_pred = tf.nn.softmax(out)
y_pred_cls = tf.argmax(out, dimension=1)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Miscellenious items finished..!!',flush=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 16, 56, 56, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 8, 28, 28, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 8, 28, 28, 256)    884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 8, 28, 28, 256)    1769728   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 4, 14, 14, 256)    0         
__________

####  Feature extraction and Distance calculation

In [4]:
## Start the session with logging placement.
init_op = tf.global_variables_initializer()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
sess.run(init_op)

## Restore model weights from previously saved model
saver = tf.train.Saver()
saver.restore(sess, os.path.join(saved_path,'activity_model.ckpt-67'))
print("Model restored from file: %s" % saved_path,flush=True)

INFO:tensorflow:Restoring parameters from /nobackup/leopauly/S2l/activity_model.ckpt-67
Model restored from file: /nobackup/leopauly/S2l/


In [5]:
## Uniform sampling of frames
def get_compress_frames_data(filename, num_frames_per_clip=16):
  ''' Given a directory containing extracted frames, return a video clip of
  (num_frames_per_clip) consecutive frames as a list of np arrays '''
  ret_arr = []
  for parent, dirnames, filenames in os.walk(filename):

    filenames = sorted(filenames)
    jump=math.floor((len(filenames)/num_frames_per_clip))
    loop=0
    for i in range(0,len(filenames),jump):
      if (loop>15):
        break
      if (filenames[i].endswith('.png')):
        image_name = str(filename) + '/' + str(filenames[i])
        img = Image.open(image_name)
        img_data = np.array(img)
        ret_arr.append(img_data)
        loop=loop+1
  ret_arr=np.array(ret_arr) #ret_arr=ret_arr/255
  return np.array(ret_arr)

In [6]:
## Extraction of features 
def extract_video_features(vid):
    vid_=vid.reshape(-1,cluster_length,height,width,channel)
    f_v = sess.graph.get_tensor_by_name('flatten_1/Reshape:0') 
    f_v_val=sess.run([f_v], feed_dict={'conv1_input:0':vid_,x_image:vid_,K.learning_phase(): 0 })
    features=np.reshape(f_v_val,(-1))
    return features

In [7]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Ch4/Leeds_Dataset_Complex_new/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Dataset/Ch4/Leeds_Dataset_Complex_new/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=np.linalg.norm(extract_video_features(vid_a)-extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('\nInter class variance bw class 1 and class 2:',inter_var)
print('Number of combinations:',np.array(dist_dif).shape)

inter1=inter_var

Sub directories: ['reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_bg_change_target', 'reach_robo_change_pos', 'reach_robo_comp0', 'reach_robo_comp1', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_comp0', 'push_robo_comp1', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']

Inter class variance bw class 1 and class 2: 295.575038791
Number of combinations: (256,)


In [8]:
## DISTANCE CALCULATION

dist_sim_class=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Ch4/Leeds_Dataset_Complex_new/Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Dataset/Ch4/Leeds_Dataset_Complex_new/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

## Calculating Distances
for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            if(sub_dir_a_!=sub_dir_b_):
                vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
                feature_dist=np.linalg.norm(extract_video_features(vid_a)-extract_video_features(vid_b))
                dist_sim_class.append(feature_dist)

            
## Calculating intra-class variance
dist_sim_class_nozero=dist_sim_class
intra_var_class=sum(dist_sim_class_nozero)/len(dist_sim_class_nozero)
print('\nInter class variance bw class :',intra_var_class)
print('Number of combinations:',np.shape(dist_sim_class_nozero))

intra=intra_var_class

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_comp0', 'push_robo_comp1', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_comp0', 'push_robo_comp1', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']

Inter class variance bw class : 288.313766797
Number of combinations: (240,)


In [9]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Ch4/Leeds_Dataset_Complex_new/Reach n Push/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')
print('Sub directories:',sub_dir_a)

base_dir_b='/nobackup/leopauly/S2l/Dataset/Ch4/Leeds_Dataset_Complex_new/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')
print('\nSub directories:',sub_dir_b)

for sub_dir_a_ in sub_dir_a:
        #print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            #print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=np.linalg.norm(extract_video_features(vid_a)-extract_video_features(vid_b))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('\nInter class variance bw class 1 and class 2:',inter_var)
print('Number of combinations:',np.array(dist_dif).shape)

inter2=inter_var

Sub directories: ['Multi_robo_M3', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_M2', 'multi_robo_arbview1', 'multi_robo_bg', 'multi_robo_change_pos', 'multi_robo_change_target', 'multi_robo_comp0', 'multi_robo_comp1', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2', 'multi_robo_sideview']

Sub directories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_comp0', 'push_robo_comp1', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']

Inter class variance bw class 1 and class 2: 319.171940207
Number of combinations: (256,)


In [10]:
##Ratios
print(round(intra/inter1,4))
print('---',round(intra/intra,4))
print(round(intra/inter2,4))

0.9754
--- 1.0
0.9033
